In [1]:
library(tidyverse)
library(repr)
library(tidymodels)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co

In [2]:
set.seed(1)
stroke_data <- read_csv("stroke_data.csv")

head(stroke_data)

Rows: 5110 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): gender, ever_married, work_type, Residence_type, bmi, smoking_status
dbl (6): id, age, hypertension, heart_disease, avg_glucose_level, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1
56669,Male,81,0,0,Yes,Private,Urban,186.21,29,formerly smoked,1


In [3]:
stroke_data_clean <- stroke_data|>
    mutate(bmi = as.numeric(bmi))|>
    filter(!is.na(bmi))|>
    mutate(stroke = factor(stroke, levels = c(0, 1), labels = c("No Stroke", "Stroke")))|>
    select(age,avg_glucose_level,bmi, hypertension, heart_disease, stroke)

head(stroke_data_clean)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `bmi = as.numeric(bmi)`.
Caused by warning:
! NAs introduced by coercion”


age,avg_glucose_level,bmi,hypertension,heart_disease,stroke
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
67,228.69,36.6,0,1,Stroke
80,105.92,32.5,0,1,Stroke
49,171.23,34.4,0,0,Stroke
79,174.12,24.0,1,0,Stroke
81,186.21,29.0,0,0,Stroke
74,70.09,27.4,1,1,Stroke


In [4]:
stroke_split <- initial_split(stroke_data_clean, prop = 0.75, strata = stroke)
stroke_train <- training(stroke_split)
stroke_test <- testing(stroke_split)

In [5]:
head(stroke_train)

age,avg_glucose_level,bmi,hypertension,heart_disease,stroke
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
67,228.69,36.6,0,1,Stroke
49,171.23,34.4,0,0,Stroke
79,174.12,24.0,1,0,Stroke
69,94.39,22.8,0,0,Stroke
81,80.43,29.7,1,0,Stroke
54,104.51,27.3,0,0,Stroke


In [6]:
stroke_irrelevant <- stroke_data_clean|>
    select(stroke, age, avg_glucose_level, bmi, hypertension, heart_disease)

stroke_subset <- stroke_irrelevant|>
    select(stroke,
               age,
               avg_glucose_level,
               bmi, hypertension, heart_disease)

names <- colnames(stroke_subset |> select(-stroke))

head(stroke_subset)
names

stroke,age,avg_glucose_level,bmi,hypertension,heart_disease
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Stroke,67,228.69,36.6,0,1
Stroke,80,105.92,32.5,0,1
Stroke,49,171.23,34.4,0,0
Stroke,79,174.12,24.0,1,0
Stroke,81,186.21,29.0,0,0
Stroke,74,70.09,27.4,1,1


[1] "age"               "avg_glucose_level" "bmi"              
[4] "hypertension"      "heart_disease"

In [7]:
stroke_recipe <- recipe(stroke ~ age + 
                       avg_glucose_level + 
                       bmi + 
                       hypertension + 
                       heart_disease, data = stroke_subset) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

In [8]:
example_formula <- paste("stroke", "~", paste(names, collapse="+"))
example_formula

[1] "stroke ~ age+avg_glucose_level+bmi+hypertension+heart_disease"

In [9]:
accuracies <- tibble(size = integer(),
                     model_string = character(),
                     accuracy = numeric())
accuracies

size,model_string,accuracy
<int>,<chr>,<dbl>


In [10]:
knn_spec <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")
knn_spec

K-Nearest Neighbor Model Specification (classification)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


In [11]:
stroke_vfold <- vfold_cv(stroke_subset, v = 5, strata = stroke)

In [12]:
n_total <- length(names)
n_total

[1] 5

In [13]:
selected <- c()
selected

NULL

In [14]:
# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("stroke", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        stroke_recipe <- recipe(as.formula(model_string),
                                data = stroke_subset) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())
         # tune the K-NN classifier with these predictors,
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(stroke_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = stroke_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |>
      add_row(size = i,
              model_string = models[[jstar]],
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
accuracies

size,model_string,accuracy
<int>,<chr>,<dbl>
1,stroke ~ avg_glucose_level,0.9574265
2,stroke ~ avg_glucose_level+bmi,0.9576302
3,stroke ~ avg_glucose_level+bmi+age,0.9574265
4,stroke ~ avg_glucose_level+bmi+age+heart_disease,0.9572229
5,stroke ~ avg_glucose_level+bmi+age+heart_disease+hypertension,0.9570192


Most accurate predictors are glucose level and bmi. 

In [15]:
stroke_vfold_k <- vfold_cv(stroke_train, v = 5, strata = stroke)

In [16]:
stroke_recipe_k <- recipe(stroke ~ 
                       avg_glucose_level + 
                       bmi, data = stroke_subset) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

In [17]:
knn_spec_k <- nearest_neighbor(weight_func = "rectangular",
                             neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

In [21]:
k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

knn_results <- workflow() |>
  add_recipe(stroke_recipe_k) |>
  add_model(knn_spec_k) |>
  tune_grid(resamples = stroke_vfold_k, grid = k_vals) |>
  collect_metrics()

accuracies_k <- knn_results |>
  filter(.metric == "accuracy")

In [22]:
best_k <- accuracies_k |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

[1] 11